In [1]:
from futu import *
quote_ctx = OpenQuoteContext(host='127.0.0.1', port=11111)
ret, data = quote_ctx.get_option_expiration_date(code='US.LI')
if ret != RET_OK:    
    print('error:', data)
dates = data
dates

2021-12-27 13:47:37,153 | 2014496 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=1, host=127.0.0.1, port=11111, user_id=7465070


,strike_time,option_expiry_date_distance,expiration_cycle
0,2021-12-31,4,N/A
1,2022-01-07,11,N/A
2,2022-01-14,18,N/A
3,2022-01-21,25,N/A
4,2022-01-28,32,N/A
5,2022-02-04,39,N/A
6,2022-02-18,53,N/A
7,2022-03-18,81,N/A
8,2022-06-17,172,N/A
9,2022-09-16,263,N/A


In [2]:
import time

code = 'US.LI'
chains = {}
for date in dates['strike_time']:
    ret, data = quote_ctx.get_option_chain(code=code, start=date, end=date)
    if ret == RET_OK:
        chains[date] = data
        print(date, ' ok')
    else:
        print(date, 'error:', data)
    time.sleep(3)

2021-12-31  ok
2022-01-07  ok
2022-01-14  ok
2022-01-21  ok
2022-01-28  ok
2022-02-04  ok
2022-02-18  ok
2022-03-18  ok
2022-06-17  ok
2022-09-16  ok
2023-01-20  ok
2024-01-19  ok


In [4]:
list(chains.values())[0].iloc[:10]

,code,name,lot_size,stock_type,option_type,stock_owner,strike_time,strike_price,suspension,stock_id,index_option_type
0,US.LI211231C20000,LI 211231 20.00C,100,DRVT,CALL,US.LI,2021-12-31,20.0,False,40636453,N/A
1,US.LI211231P20000,LI 211231 20.00P,100,DRVT,PUT,US.LI,2021-12-31,20.0,False,40629724,N/A
2,US.LI211231C21000,LI 211231 21.00C,100,DRVT,CALL,US.LI,2021-12-31,21.0,False,41135745,N/A
3,US.LI211231P21000,LI 211231 21.00P,100,DRVT,PUT,US.LI,2021-12-31,21.0,False,41152450,N/A
4,US.LI211231C22000,LI 211231 22.00C,100,DRVT,CALL,US.LI,2021-12-31,22.0,False,41114898,N/A
5,US.LI211231P22000,LI 211231 22.00P,100,DRVT,PUT,US.LI,2021-12-31,22.0,False,41193914,N/A
6,US.LI211231C23000,LI 211231 23.00C,100,DRVT,CALL,US.LI,2021-12-31,23.0,False,41204490,N/A
7,US.LI211231P23000,LI 211231 23.00P,100,DRVT,PUT,US.LI,2021-12-31,23.0,False,41133069,N/A
8,US.LI211231C24000,LI 211231 24.00C,100,DRVT,CALL,US.LI,2021-12-31,24.0,False,41114134,N/A
9,US.LI211231P24000,LI 211231 24.00P,100,DRVT,PUT,US.LI,2021-12-31,24.0,False,41121325,N/A


In [52]:
code = 'LI'
proxy = 'https://oracle.ihuhao.com:443'
base_url = 'https://query1.finance.yahoo.com/v7/finance/quote?lang=en-US\&region=US\&corsDomain=finance.yahoo.com'
url = base_url + '\&fields=regularMarketPrice\&symbols=' + code
data = !curl -s -x $proxy $url
import json
data = json.loads(data[0])
data

{'quoteResponse': {'result': [{'language': 'en-US',
    'region': 'US',
    'quoteType': 'EQUITY',
    'quoteSourceName': 'Delayed Quote',
    'triggerable': True,
    'marketState': 'PREPRE',
    'sourceInterval': 15,
    'exchangeDataDelayedBy': 0,
    'tradeable': False,
    'firstTradeDateMilliseconds': 1596115800000,
    'priceHint': 2,
    'regularMarketTime': 1640293202,
    'regularMarketPrice': 30.42,
    'fullExchangeName': 'NasdaqGS',
    'exchange': 'NMS',
    'exchangeTimezoneName': 'America/New_York',
    'exchangeTimezoneShortName': 'EST',
    'gmtOffSetMilliseconds': -18000000,
    'market': 'us_market',
    'esgPopulated': False,
    'symbol': 'LI'}],
  'error': None}}

In [58]:
stock_price = data['quoteResponse']['result'][0]['regularMarketPrice']
stock_price

30.42

In [67]:
import numpy as np
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

In [70]:
options = list(chains.values())[0]
at_the_money = find_nearest(options['strike_price'].unique(), stock_price)
at_the_money

30.0

In [75]:
atm_call = options[(options['strike_price']==at_the_money) & (options['option_type']=='CALL')]
atm_put = options[(options['strike_price']==at_the_money) & (options['option_type']=='PUT')]
atm_call.iloc[0].code, atm_put.iloc[0].code

('US.LI211231C30000', 'US.LI211231P30000')

In [86]:
atm_codes = []
for date, options in chains.items():
    at_the_money = find_nearest(options['strike_price'].unique(), stock_price)
    atm_call = options[(options['strike_price']==at_the_money) & (options['option_type']=='CALL')]
    atm_put = options[(options['strike_price']==at_the_money) & (options['option_type']=='PUT')]
    atm_codes.append([atm_call.iloc[0].code, atm_put.iloc[0].code])
    print(date, atm_call.iloc[0].code, atm_put.iloc[0].code)

2021-12-31 US.LI211231C30000 US.LI211231P30000
2022-01-07 US.LI220107C30000 US.LI220107P30000
2022-01-14 US.LI220114C30000 US.LI220114P30000
2022-01-21 US.LI220121C30000 US.LI220121P30000
2022-01-28 US.LI220128C30000 US.LI220128P30000
2022-02-04 US.LI220204C30000 US.LI220204P30000
2022-02-18 US.LI220218C30000 US.LI220218P30000
2022-03-18 US.LI220318C30000 US.LI220318P30000
2022-06-17 US.LI220617C30000 US.LI220617P30000
2022-09-16 US.LI220916C30000 US.LI220916P30000
2023-01-20 US.LI230120C30000 US.LI230120P30000
2024-01-19 US.LI240119C30000 US.LI240119P30000


In [87]:
ret, data = quote_ctx.get_market_snapshot([code for c_p in atm_codes for code in c_p])
if ret != RET_OK:
    print('error:', data)

In [90]:
atm_options = data

In [91]:
atm_options.head()

,code,update_time,last_price,open_price,high_price,low_price,prev_close_price,volume,turnover,turnover_rate,...,pre_turnover,pre_change_val,pre_change_rate,pre_amplitude,after_price,after_high_price,after_low_price,after_change_val,after_change_rate,after_amplitude
0,US.LI211231C30000,2021-12-23 16:00:00,1.12,1.08,1.30,0.98,1.29,141,16632.0,0.0,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
1,US.LI211231P30000,2021-12-23 16:00:00,0.65,0.93,1.00,0.62,0.89,118,8802.0,0.0,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
2,US.LI220107C30000,2021-12-23 16:00:00,1.89,1.55,1.89,1.55,1.65,2,344.0,0.0,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
3,US.LI220107P30000,2021-12-23 16:00:00,1.21,1.37,1.37,1.21,2.76,5,647.0,0.0,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
4,US.LI220114C30000,2021-12-23 16:00:00,1.62,1.62,0.00,0.00,1.62,0,0.0,0.0,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A


假设欧式期权，标的资产不付红利，现在有两个不同组合。

A组合：看涨期权c+现金X/(1+r(T-t))

B组合：看跌期权p+标的资产S

In [118]:
def C_X(C, X, t, r=0.03):
    return C + X / (1 + r*t/365)
def P_S(P, S):
    return P + S

In [119]:
data = []
for call_code, put_code in atm_codes:
    C = atm_options[atm_options['code']==call_code].iloc[0]
    P = atm_options[atm_options['code']==put_code].iloc[0]
    C_value = C.last_price * C.option_contract_multiplier
    X_value = C.option_strike_price * C.option_contract_multiplier
    P_value = P.last_price * P.option_contract_multiplier
    S_value = stock_price * P.option_contract_multiplier    
    A = C_X(C_value, X_value, C.option_expiry_date_distance)
    B = P_S(P_value, S_value)
    if B > A:
        # 买入看涨，卖出看跌和标的
        cost = -C_value + P_value + S_value
    else:
        # 卖出看涨，买入看跌和标的
        cost = C_value - P_value - S_value
    data.append([C.option_expiry_date_distance,C.strike_time, C.option_strike_price, C.last_price, P.last_price, 
                 A, B, A-B, cost, abs((A-B)/cost)])
df = pd.DataFrame(data, columns=['expiry_day', 'strike_time', 'strike_price', 'call_price', 'put_price', 
                                 'C+X', 'P+S', 'profit', 'cost', 'rate'])
df    

,expiry_day,strike_time,strike_price,call_price,put_price,C+X,P+S,profit,cost,rate
0,4,2021-12-31,30.0,1.12,0.65,3111.014023,3107.0,4.014023,-2995.0,0.001340
1,11,2022-01-07,30.0,1.89,1.21,3186.290121,3163.0,23.290121,-2974.0,0.007831
2,18,2022-01-14,30.0,1.62,1.50,3157.568200,3192.0,-34.431800,3030.0,0.011364
3,25,2022-01-21,30.0,2.18,1.60,3211.848257,3202.0,9.848257,-2984.0,0.003300
4,32,2022-01-28,30.0,2.50,3.60,3242.130287,3402.0,-159.869713,3152.0,0.050720
5,39,2022-02-04,30.0,0.00,0.00,2990.414288,3042.0,-51.585712,3042.0,0.016958
6,53,2022-02-18,30.0,2.93,2.51,3279.988188,3293.0,-13.011812,3000.0,0.004337
7,81,2022-03-18,30.0,3.65,3.30,3345.159486,3372.0,-26.840514,3007.0,0.008926
8,172,2022-06-17,30.0,4.80,5.50,3438.180246,3592.0,-153.819754,3112.0,0.049428
9,263,2022-09-16,30.0,6.35,6.50,3571.522835,3692.0,-120.477165,3057.0,0.039410
